In [1]:
import (
	"encoding/json"
    . "github.com/FactomProject/ptnet-eventstore/contract"
	"github.com/FactomProject/ptnet-eventstore/finite"
	. "github.com/FactomProject/ptnet-eventstore/identity"
	. "github.com/FactomProject/ptnet-eventstore/ptnet"
	"github.com/FactomProject/ptnet-eventstore/x"
)

In [2]:
m := Machine{
	//                   00 01 02 10 11 12 20 21 22 O  X $O $X $DEP
	Initial: StateVector{1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1},

	Transitions: map[string]Transition{

		//                00 01 02 10 11 12 20 21 22  O  X $O $X $DEP
		BEGIN: Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, -1, -1, -1},

		"X00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, 1, -1, 0, 0, 0},
		"X12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, 1, -1, 0, 0, 0},
		"X20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, 1, -1, 0, 0, 0},
		"X21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, 1, -1, 0, 0, 0},
		"X22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, 1, -1, 0, 0, 0},

		"O00": Transition{-1, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O01": Transition{0, -1, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O02": Transition{0, 0, -1, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O10": Transition{0, 0, 0, -1, 0, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O11": Transition{0, 0, 0, 0, -1, 0, 0, 0, 0, -1, 1, 0, 0, 0},
		"O12": Transition{0, 0, 0, 0, 0, -1, 0, 0, 0, -1, 1, 0, 0, 0},
		"O20": Transition{0, 0, 0, 0, 0, 0, -1, 0, 0, -1, 1, 0, 0, 0},
		"O21": Transition{0, 0, 0, 0, 0, 0, 0, -1, 0, -1, 1, 0, 0, 0},
		"O22": Transition{0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, 0, 0, 0},

		"WINX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1, 0}, // Depositor acts as judge
		"WINO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 1, 0, 0},
		"ENDX": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 1},
		"ENDO": Transition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 1},
	},
}

In [3]:
gameContract := Declaration{ // array of inputs/outputs also referenced by guards and conditions
	Inputs: []AddressAmountMap{ // array of input depositors
		AddressAmountMap{Address[DEPOSITOR], 1},
	},
	Outputs: []AddressAmountMap{ // array of possible redeemers
		AddressAmountMap{Address[DEPOSITOR], 1},
		AddressAmountMap{Address[PLAYERX], 1},
		AddressAmountMap{Address[PLAYERO], 1},
	},
	BlockHeight: 60221409,      // deadline for halting state
	Salt:        "|RANDOM|",    // added random salt
	ContractID:  "|OctoeContractID|",   // unique ID for this contract instance
	Schema:      "octoe-v1",    // versioned contract schema
	State:       m.Initial,     // state machine initial state
	Actions:     m.Transitions, // state machine defined transitions
	Guards: []Condition{ // guard clause restricts actions
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP // variable labels
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0},  // Admin - 'contract owner' can take action at any time
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0}, // PlayerX - players must move only when it's their turn
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0}, // PlayerO
	},
	Conditions: []Condition{ // contract conditions specify additional redeem conditions
		//       00 01 02 10 11 12 20 21 22  O  X $O $X $DEP  // variable labels
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1}, // game ended without winner tokens are unlocked for original depositor
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0}, // game ended PlayerX wins
		Condition{0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -1, 0, 0}, // game ended PlayerO wins
	},
}

In [4]:
offer := finite.Offer{
	Declaration: gameContract,
}

In [5]:
[]interface{}{gameContract.ContractID, gameContract.Schema, gameContract.BlockHeight}

[|OctoeContractID| octoe-v1 60221409]

In [6]:
Exists(gameContract.Schema, gameContract.ContractID)

false

In [7]:
txn := finite.OfferTransaction(offer, Private[DEPOSITOR])
[]interface{}{txn.Action, txn.InputState, "=>", txn.OutputState}

[EXEC [1 1 1 1 1 1 1 1 1 1 1 1 1 1] => [1 1 1 1 1 1 1 1 1 0 1 0 0 0]]

In [8]:
Exists(gameContract.Schema, optionContract.ContractID)

ERROR: repl.go:1:29: undefined identifier: optionContract

In [9]:
IsHalted(gameContract)

false

In [10]:
CanRedeem(gameContract, Public[DEPOSITOR])

false

In [11]:
func commit(action string, key PrivateKey, payload []byte) (finite.Transaction, error) {
	pub := PublicKey{}
	copy(pub[:], x.PrivateKeyToPub(key[:]))
	return finite.ExecuteTransaction(finite.Execution{
		Command: Command{
			ChainID:    CHAIN_ID, // test values
			ContractID: "|OctoeContractID|",
			Schema:     OctoeV1, // state machine version
			Action:     action,   // state machine action
			Amount:     1,        // triggers input action 'n' times
			Payload:    payload,      // arbitrary data optionally included
			Pubkey:     pub,
		},
	}, key)
}

In [12]:
payload, _ := json.Marshal([]string{"hello", "world"})
event1, _ := commit("X11", Private[PLAYERX], payload)

[]interface{}{event1.Action, event1.InputState, "=>", event1.OutputState}

[X11 [1 1 1 1 1 1 1 1 1 0 1 0 0 0] => [1 1 1 1 0 1 1 1 1 1 0 0 0 0]]

In [13]:
// state machine makes players take turns
event2, err := commit("X00", Private[PLAYERX], nil)
[]interface{}{event2.Action, event2.InputState, "=>", err}

[X00 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => invalid output: -1 offset: 10]

In [14]:
// contract makes players sign each event - here we try to sign with an incorrect key
event3, err := commit("O01", Private[PLAYERX], nil)

[]interface{}{event3.Action, event3.InputState, "=>", err}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => failed guard condition]

In [15]:
event4, err := commit("O01", Private[PLAYERO], nil)

[]interface{}{event4.Action, event4.InputState, "=>", event4.OutputState}

[O01 [1 1 1 1 0 1 1 1 1 1 0 0 0 0] => [1 0 1 1 0 1 1 1 1 0 1 0 0 0]]

In [16]:
// state machine ensures move is used only once
event5, err := commit("X11", Private[PLAYERX], nil)

[]interface{}{event5.Action, event5.InputState, "=>", err}

[X11 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => invalid output: -1 offset: 4]

In [17]:
event6, err := commit("X00", Private[PLAYERX], nil)

[]interface{}{event6.Action, event6.InputState, "=>", event6.OutputState}

[X00 [1 0 1 1 0 1 1 1 1 0 1 0 0 0] => [0 0 1 1 0 1 1 1 1 1 0 0 0 0]]

In [18]:
event7, err := commit("O02", Private[PLAYERO], nil)

[]interface{}{event7.Action, event7.InputState, "=>", event7.OutputState}

[O02 [0 0 1 1 0 1 1 1 1 1 0 0 0 0] => [0 0 0 1 0 1 1 1 1 0 1 0 0 0]]

In [19]:
event8, err := commit("X22", Private[PLAYERX], nil)

[]interface{}{event8.Action, event8.InputState, "=>", event8.OutputState}

[X22 [0 0 0 1 0 1 1 1 1 0 1 0 0 0] => [0 0 0 1 0 1 1 1 0 1 0 0 0 0]]

In [20]:
// contract depositor confirms the winner
event9, err := commit("WINX", Private[DEPOSITOR], nil)

[]interface{}{event9.Action, event9.InputState, "=>", event9.OutputState}

[WINX [0 0 0 1 0 1 1 1 0 1 0 0 0 0] => [0 0 0 1 0 1 1 1 0 0 0 0 1 0]]

In [21]:
// contract is only Redeemable after Halting
IsHalted(gameContract)

true

In [22]:
// only the winner can redeem
CanRedeem(gameContract, Public[PLAYERX])

true

In [23]:
CanRedeem(gameContract, Public[PLAYERO])

false